In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from prophet import Prophet
from sklearn.model_selection import train_test_split

file_path = r"D:\Downloads\Germany_Simplified_Data_Short.csv"
data = pd.read_csv(file_path)
data['Datetime'] = pd.to_datetime(data['Datetime'], format='%d/%m/%Y %H:%M')
data = data.rename(columns={"Datetime": "ds", "Price (EUR/MWHE)": "y"})
data.set_index('ds', inplace=True)

train_data, test_data = train_test_split(data, test_size=0.66, shuffle=False)

model = Prophet(yearly_seasonality=True, weekly_seasonality=True, daily_seasonality=True,
                seasonality_mode='additive', changepoint_prior_scale=0.01,
                seasonality_prior_scale=1.0, interval_width=0.1)
model.add_seasonality(name='monthly', period=30.5, fourier_order=5)
model.fit(train_data.reset_index())

future = model.make_future_dataframe(periods=len(test_data), freq='H')
forecast = model.predict(future)

predictions_df = pd.DataFrame({
    'Datetime': test_data.index,
    'Actual_Price': test_data['y'].values,
    'Prophet_Predicted': forecast['yhat'].iloc[-len(test_data):].values,
})

output_path = r"D:\Downloads\Prophet_Predictions.csv"
predictions_df.to_csv(output_path, index=False)

plt.figure(figsize=(12, 6))
plt.plot(test_data.index, test_data['y'], label="Actual Price")
plt.plot(test_data.index, forecast['yhat'].iloc[-len(test_data):], label="Prophet Forecast")
plt.legend()
plt.xlabel("Time")
plt.ylabel("Price")
plt.title("Comparison of Prediction Models (Prophet)")
plt.grid()
plt.show()

start_date = '09/09/2023 00:00'
end_date = '13/09/2023 00:00'
date_mask = (predictions_df['Datetime'] >= start_date) & (predictions_df['Datetime'] <= end_date)
filtered_df = predictions_df[date_mask]
filtered_df['Datetime'] = pd.to_datetime(filtered_df['Datetime'])

plt.figure(figsize=(12, 6))
plt.plot(filtered_df['Datetime'], filtered_df['Actual_Price'], label="Actual Price", color="blue")
plt.plot(filtered_df['Datetime'], filtered_df['Prophet_Predicted'], label="Prophet Forecast", linestyle="--", color="orange")
plt.legend()
plt.xlabel("Datetime")
plt.ylabel("Price")
plt.title("Prophet Forecast vs Actual Prices")
plt.grid()
plt.show()